This script use kirita data to annotate the cellbender-corrected count

1. Tacco integration

- Initial Date: 12/8/2025

In [6]:
# Set 'Liberation Sans' as the default font
from matplotlib import rcParams
rcParams['font.family'] = 'Liberation Sans'

In [7]:
#!/usr/bin/env python
"""
Standard Scanpy Analysis Pipeline for D0_1 and D3_1 samples
Following: https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering.html

No integration step - just standard preprocessing, QC, normalization, 
dimensionality reduction, and clustering.
"""

import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os

# Set scanpy settings
sc.settings.set_figure_params(dpi=200, facecolor='white', figsize=(6, 5))
sc.settings.verbosity = 3

# =============================================================================
# CONFIGURATION - Update these paths
# =============================================================================
# Input: Combined h5ad file with D0_1 and D3_1 samples
INPUT_H5AD = "/home/data3/dianli/projects/kenji_rhabdo_2025/processed_data/cellbender_count.h5ad"

ref_H5AD = "/home/data3/dianli/projects/kenji_rhabdo_2025/processed_data/kirita_126578_cell_raw_count.h5ad"
# Output directory
OUTPUT_DIR = "/home/data3/dianli/projects/kenji_rhabdo_2025/processed_data/analysis/cellbender_count/label_transfer/tacco/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Sample column name
SAMPLE_COL = "sample"  # or "sample" depending on your data

# Samples to subset
SAMPLES_TO_USE = ["D0_1", "D3_1"]

# =============================================================================
# 1. LOAD DATA AND SUBSET TO D0_1 AND D3_1
# =============================================================================
print("=" * 60)
print("1. LOADING AND SUBSETTING DATA")
print("=" * 60)

# Load the query adata
adata_query = sc.read_h5ad(INPUT_H5AD)
print(f"Loaded adata: {adata_query.shape[0]} cells x {adata_query.shape[1]} genes")

# Load the ref adata
adata_ref = sc.read_h5ad(ref_H5AD)
print(f"Loaded adata: {adata_ref.shape[0]} cells x {adata_ref.shape[1]} genes")

# 1. Ensure your data is loaded (example placeholders)
# adata_query = sc.read("query_data.h5ad")
# adata_ref = sc.read("reference_data.h5ad")

1. LOADING AND SUBSETTING DATA
Loaded adata: 22612 cells x 32285 genes
Loaded adata: 126578 cells x 27133 genes


In [8]:
adata_query.obs

,sample,in_original,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_SCT,nFeature_SCT,pANN_0.25_0.26_600,DF.classifications_0.25_0.26_600,...,Phase,old.ident,SCT_snn_res.0.8,seurat_clusters,SCT.weight,wsnn_res.0.6,wsnn_res.0.4,nCount_TSSaccessibility,nFeature_TSSaccessibility,celltype
TCGTTAGCAACTAGAA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TAGCCTGAGGAAGTAT-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTGTCCAAGGAGGTTA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTAAGCGCAACGTGCT-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GGATATTGTGGTGAGA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCTAGCCTCCATCAGG-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CCTAAAGGTACCGTAA-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GATTCGCCACAGACTC-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TTTGGTAAGCTGGAAA-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
adata_ref.obs

,orig.ident,nCount_RNA,nFeature_RNA,rep,percent.mito,percent.Rpl,percent.Rps,time,seurat_clusters,batch.n,rep.n,S.Score,G2M.Score,Phase,CC.Difference,AllIDn1,AllIDn2,name,Replicates,celltype
IRIsham1b1_AAACCTGAGCTAACAA,IRIsham1b1,1412,600,IRIsham1,0.0,0.000000,0.000000,Control,0,1,1,-0.055359,-0.012965,G1,-0.042393,1,1,PTS1,1_1,PTS1
IRIsham1b1_AAACCTGCATCGGGTC,IRIsham1b1,2134,603,IRIsham1,0.0,0.000000,0.046860,Control,0,1,1,-0.032639,0.025136,G2M,-0.057775,1,1,PTS1,1_1,PTS1
IRIsham1b1_AAACCTGGTAACGCGA,IRIsham1b1,2829,697,IRIsham1,0.0,0.000000,0.000000,Control,1,1,1,-0.009040,-0.042371,G1,0.033331,2,2,PTS2,1_1,PTS2
IRIsham1b1_AAACCTGGTAGCGCAA,IRIsham1b1,2216,756,IRIsham1,0.0,0.000000,0.000000,Control,1,1,1,-0.018934,0.043305,G2M,-0.062239,2,2,PTS2,1_1,PTS2
IRIsham1b1_AAACCTGGTAGGGACT,IRIsham1b1,2692,685,IRIsham1,0.0,0.000000,0.037147,Control,0,1,1,0.066002,-0.054710,S,0.120712,1,1,PTS1,1_1,PTS1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IRI6w3_TTTGTCATCACCTCGT,IRI6w3,1521,384,IRI6w3,0.0,0.000000,0.000000,6weeks,11,4,3,0.020728,0.008541,S,0.012188,15,14,CNT,3,CNT
IRI6w3_TTTGTCATCCAGAGGA,IRI6w3,1749,382,IRI6w3,0.0,0.000000,0.000000,6weeks,20,4,3,-0.017944,-0.007951,G1,-0.009993,14,13,DCT-CNT,3,DCT-CNT
IRI6w3_TTTGTCATCCGCAGTG,IRI6w3,1462,448,IRI6w3,0.0,0.000000,0.000000,6weeks,17,4,3,-0.032382,-0.011710,G1,-0.020672,10,9,CTAL1,3,CTAL1
IRI6w3_TTTGTCATCTGATTCT,IRI6w3,1077,426,IRI6w3,0.0,0.185701,0.000000,6weeks,3,4,3,0.043493,-0.041505,S,0.084999,8,7,MTAL,3,MTAL


In [10]:
import tacco as tc
from datetime import datetime

print(datetime.now())
# 2. Run the annotation transfer
# This uses Optimal Transport (OT) by default to handle platform effects
tc.tl.annotate(
    adata_query, 
    reference=adata_ref, 
    annotation_key='celltype',  # The column in adata_ref.obs containing the labels
    result_key='tacco_celltype', # Where to save the output in adata_query
    method='OT'                  # 'OT' is recommended; alternatives: 'svm', 'tangram'
)

print(datetime.now())

2025-12-08 17:04:57.012807
Starting preprocessing
Annotation profiles were not found in `reference.varm["celltype"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 9.49 seconds.
Starting annotation of data with shape (22586, 23904) and a reference of shape (126578, 23904) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=celltype, normalize_to=adata
   +- multi center: multi_center=None multi_center_amplitudes=True
      +- bisection boost: bisections=4, bisection_divisor=3
         +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  1.3507226601885356 28.921174472880665
bisection run on 1
bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 29.11 seconds.
2025-12-08 17:05:36.588682

In [11]:
print(datetime.now())

# Convert probabilities to a single "Hard" label
tc.utils.get_maximum_annotation(
    adata_query, 
    obsm_key='tacco_celltype',        # The key in .obsm with probabilities
    result_key='tacco_celltype'   # New column name in .obs for the label
)

print(datetime.now())

2025-12-08 17:05:36.593410
2025-12-08 17:05:36.597527


In [12]:
adata_query.obs

,sample,in_original,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_SCT,nFeature_SCT,pANN_0.25_0.26_600,DF.classifications_0.25_0.26_600,...,old.ident,SCT_snn_res.0.8,seurat_clusters,SCT.weight,wsnn_res.0.6,wsnn_res.0.4,nCount_TSSaccessibility,nFeature_TSSaccessibility,celltype,tacco_celltype
TCGTTAGCAACTAGAA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2
TAGCCTGAGGAAGTAT-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2
GTGTCCAAGGAGGTTA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2
GTAAGCGCAACGTGCT-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS1
GGATATTGTGGTGAGA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCTAGCCTCCATCAGG-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EC1
CCTAAAGGTACCGTAA-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tcell
GATTCGCCACAGACTC-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tcell
TTTGGTAAGCTGGAAA-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EC1


In [13]:
# Alternative: just use idxmax
# adata_query.obs['predicted_celltype'] = adata_query.obsm['tacco_celltype'].idxmax(axis=1)
adata_query.obs['tacco_confidence'] = adata_query.obsm['tacco_celltype'].max(axis=1)

In [14]:
adata_query.obs

,sample,in_original,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,nCount_SCT,nFeature_SCT,pANN_0.25_0.26_600,DF.classifications_0.25_0.26_600,...,SCT_snn_res.0.8,seurat_clusters,SCT.weight,wsnn_res.0.6,wsnn_res.0.4,nCount_TSSaccessibility,nFeature_TSSaccessibility,celltype,tacco_celltype,tacco_confidence
TCGTTAGCAACTAGAA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2,0.535082
TAGCCTGAGGAAGTAT-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2,0.279399
GTGTCCAAGGAGGTTA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2,0.466945
GTAAGCGCAACGTGCT-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS1,0.471487
GGATATTGTGGTGAGA-1_1,D0_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PTS2,0.201327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCTAGCCTCCATCAGG-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EC1,0.803058
CCTAAAGGTACCGTAA-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tcell,0.400570
GATTCGCCACAGACTC-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tcell,0.567306
TTTGGTAAGCTGGAAA-1_2,D3_1,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EC1,0.474707


### export tacco labels

In [15]:
# Save the predicted labels and confidence scores
labels_df = adata_query.obs[['tacco_celltype', 'tacco_confidence']].copy()
labels_df.to_csv(os.path.join(OUTPUT_DIR, 'tacco_labels.csv'))